In [ ]:
import tensorflow as tf

from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import numpy as np
import random

In [ ]:
#### Import data from google co-lab ###
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
from numpy import load
import os
# load the face dataset
data = np.load("/content/gdrive/MyDrive/Colab Notebooks/npz/celeba_25K.npz")
print(data)
faces = data["arr_0"]
print("Loaded: ", faces.shape)




Loaded:  (25000, 80, 80, 3)


In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(80,80,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(32, (5,5), padding="same", input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 40x40
    model.add(Conv2D(64, (5,5), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 20x30
    model.add(Conv2D(128, (5,5), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 10x10
    model.add(Conv2D(128, (5,5), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 5x5
    model.add(Conv2D(256, (5,5), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation="sigmoid"))
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

disc_model = define_discriminator()
# visualize discriminator model parameters
print(disc_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 80, 80, 32)        2432      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 80, 80, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 64)        51264     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 40, 40, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       204928    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 20, 20, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 128)       4

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 5x5 feature maps
    n_nodes = 128 * 5 * 5
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((5, 5, 128)))
    # upsample to 10x10
    model.add(Conv2DTranspose(16, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 20x20
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 40x40
    model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 80x80
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    # output layer 80x80x3
    model.add(Conv2D(3, (5,5), activation="tanh", padding="same"))
    return model

gen_model = define_generator(latent_dim=100)
gen_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 3200)              323200    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 3200)              0         
                                                                 
 reshape (Reshape)           (None, 5, 5, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 10, 10, 16)       32784     
 nspose)                                                         
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 10, 10, 16)        0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 20, 20, 32)       8224      
 ranspose)                                            

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss="binary_crossentropy", optimizer=opt)
    return model

demo_gan_model = define_gan(gen_model, disc_model)
print(demo_gan_model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 80, 80, 3)         537843    
                                                                 
 sequential (Sequential)     (None, 1)                 1494209   
                                                                 
Total params: 2,032,052
Trainable params: 537,843
Non-trainable params: 1,494,209
_________________________________________________________________
None


In [ ]:
# load and prepare training images
def load_real_samples():
    # load the face dataset
    #data = load("img_align_celeba.npz")
    X = data["arr_0"]
    # convert from unsigned ints to floats
    X = X.astype("float32")
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate real class labels (1)
    y = np.ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create ✬fake✬ class labels (0)
    y = np.zeros((n_samples, 1))
    return X, y

In [ ]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print(">Accuracy real: %.0f%%, fake: %.0f%%" % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = "/content/gdrive/MyDrive/Colab Notebooks/GAN_model/celebA_generator_model_%03d.h5" % (epoch+1)
    g_model.save(filename)


# create and save a plot of generated images
def save_plot(examples, epoch, n=10):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis("off")
        # plot raw pixel data
        plt.imshow(examples[i])
    # save plot to file
    filename = "/content/gdrive/MyDrive/Colab Notebooks/GAN_model/celebA_generated_plot_e%03d.png" % (epoch+1)
    plt.savefig(filename)
    plt.close()


In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected real samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate ✬fake✬ examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print("Epcoch>%d, Batch %d/%d, d1=%.3f, d2=%.3f g=%.3f" %
                  (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes

        if (i + 1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

2/2 [==============================] - 0s 17ms/step
Epcoch>1, Batch 1/195, d1=0.709, d2=0.695 g=0.693
2/2 [==============================] - 0s 12ms/step
Epcoch>1, Batch 2/195, d1=0.576, d2=0.696 g=0.692
2/2 [==============================] - 0s 14ms/step
Epcoch>1, Batch 3/195, d1=0.335, d2=0.708 g=0.682
2/2 [==============================] - 0s 13ms/step
Epcoch>1, Batch 4/195, d1=0.054, d2=0.735 g=0.671
2/2 [==============================] - 0s 12ms/step
Epcoch>1, Batch 5/195, d1=0.006, d2=0.748 g=0.681
2/2 [==============================] - 0s 12ms/step
Epcoch>1, Batch 6/195, d1=0.005, d2=0.721 g=0.705
2/2 [==============================] - 0s 12ms/step
Epcoch>1, Batch 7/195, d1=0.020, d2=0.695 g=0.722
2/2 [==============================] - 0s 12ms/step
Epcoch>1, Batch 8/195, d1=0.020, d2=0.682 g=0.746
2/2 [==============================] - 0s 12ms/step
Epcoch>1, Batch 9/195, d1=0.008, d2=0.649 g=0.818
2/2 [==============================] - 0s 12ms/step
Epcoch>1, Batch 10/195, d1=0.0